In [43]:
import numpy as np

import Go_Board
import time
import random
import MCTS_GO

from sgfmill import sgf

import pandas as pd

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
X = []
Y = []

In [28]:
dataset_dir = './go9-large/'#dataset/sgf/train/0000/'

from os import listdir
from os.path import isfile, join
examples = [f for f in listdir(dataset_dir) if isfile(join(dataset_dir, f))]

#example = '2014-12-12T14:25:20.109Z_ls125rvjkot8.sgf'
board_size = 9
examples = examples[:100]

In [29]:
for example in examples:
    board_go = Go_Board.Our_Board(board_size=board_size)
    with open(dataset_dir+example, "rb") as f:
        game = sgf.Sgf_game.from_bytes(f.read())
    winner = game.get_winner()
    board_size = game.get_size()
    root_node = game.get_root()

    board = np.zeros(shape=(board_size,board_size),dtype=int)

    for node in game.get_main_sequence():

        if node.get_move()[0] is not None:
            move = node.get_move()
            player = move[0]
            coord = (move[1][1],move[1][0]) if move[1] is not None else (-1,-1)

            board  = board_go.get_matrix()

            y = np.zeros(shape=(board_size,board_size),dtype=int)
            if(coord[0] != -1 and coord[1] != -1):
                y[coord[0]][coord[1]] = 1
                y = y.reshape(board_size*board_size)
                y = np.append(y,[0])
            else:
                y = np.append(y,[1])

            x = board * -1 if player == 'w' else board
            board_go.push(coord)
            X.append(x)
            Y.append(y)

In [34]:
np.reshape(X,(board_size,board_size,len(X))).shape

(9, 9, 355190)

In [35]:
np.shape(X)

(355190, 9, 9)

In [36]:
np.shape(Y)

(355190, 82)

In [41]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [39]:
df

,0,1,2,3,4,5,6,7,8,9,...,355180,355181,355182,355183,355184,355185,355186,355187,355188,355189
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, -1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, -1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, ...",...,"[[0, 0, 0, 0, -1, 0, 0, 0, 0], [0, -1, 0, 1, -...","[[0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, -1, 1,...","[[0, 0, 0, 0, -1, 0, 0, 0, 0], [0, -1, 0, 1, -...","[[0, 0, 0, 0, 1, -1, 0, 0, 0], [0, 1, 0, -1, 1...","[[0, 0, 0, -1, -1, 1, 0, 0, 0], [0, -1, 0, 1, ...","[[0, 0, 0, 1, 1, -1, 0, 0, 0], [0, 1, -1, -1, ...","[[0, 0, 0, -1, -1, 1, 0, 0, 0], [0, -1, 1, 1, ...","[[0, 0, 0, 1, 1, -1, 0, 0, 0], [0, 1, -1, -1, ...","[[0, 0, -1, -1, -1, 1, 0, 0, 0], [0, -1, 1, 1,...","[[0, -1, 0, 0, 0, -1, 0, 0, 0], [0, 1, -1, -1,..."
